<a href="https://colab.research.google.com/github/shraav2907/imlo_coursework/blob/main/imlo_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import Flowers102
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score

class FlowerNet(nn.Module):
    def __init__(self):
        super(FlowerNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 28 * 28, 128)  # Updated for the additional conv layer
        self.fc2 = nn.Linear(128, 102)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 64 * 28 * 28)  # Updated for the additional conv layer
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

model = FlowerNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = Flowers102(root='./data', split='train', transform=train_transform, download=True)

val_test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

batch_size = 32
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in data_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(data_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

def validate_model(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    val_loss /= len(val_loader.dataset)
    val_accuracy = correct / total
    precision = precision_score(all_labels, all_predictions, average='weighted')
    recall = recall_score(all_labels, all_predictions, average='weighted')
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2%}")
    print(f"Validation Precision: {precision:.2%}, Validation Recall: {recall:.2%}")

def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    test_loss /= len(test_loader.dataset)
    test_accuracy = correct / total
    precision = precision_score(all_labels, all_predictions, average='weighted')
    recall = recall_score(all_labels, all_predictions, average='weighted')
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2%}")
    print(f"Test Precision: {precision:.2%}, Test Recall: {recall:.2%}")

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_dataset = Flowers102(root='./data', split='val', transform=val_transform, download=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Validate the model
validate_model(model, val_loader, criterion)

# Load the Flowers102 test dataset
test_dataset = Flowers102(root='./data', split='test', transform=test_transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Test the model
test_model(model, test_loader, criterion)


Using device: cuda


100%|██████████| 344862509/344862509 [00:04<00:00, 69584137.54it/s] 


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 1895176.06it/s]


100%|██████████| 14989/14989 [00:00<00:00, 42221909.10it/s]


Epoch [1/200], Loss: 4.6534
Epoch [2/200], Loss: 4.5616
Epoch [3/200], Loss: 4.3976
Epoch [4/200], Loss: 4.2773
Epoch [5/200], Loss: 4.1888
Epoch [6/200], Loss: 4.0831
Epoch [7/200], Loss: 3.9974
Epoch [8/200], Loss: 3.9743
Epoch [9/200], Loss: 3.8794
Epoch [10/200], Loss: 3.8447
Epoch [11/200], Loss: 3.7598
Epoch [12/200], Loss: 3.7176
Epoch [13/200], Loss: 3.6605
Epoch [14/200], Loss: 3.6049
Epoch [15/200], Loss: 3.5705
Epoch [16/200], Loss: 3.5597
Epoch [17/200], Loss: 3.5737
Epoch [18/200], Loss: 3.4499
Epoch [19/200], Loss: 3.4578
Epoch [20/200], Loss: 3.3621
Epoch [21/200], Loss: 3.3092
Epoch [22/200], Loss: 3.3262
Epoch [23/200], Loss: 3.2937
Epoch [24/200], Loss: 3.2107
Epoch [25/200], Loss: 3.2959
Epoch [26/200], Loss: 3.1344
Epoch [27/200], Loss: 3.0944
Epoch [28/200], Loss: 3.0553
Epoch [29/200], Loss: 3.0160
Epoch [30/200], Loss: 3.0521
Epoch [31/200], Loss: 3.0022
Epoch [32/200], Loss: 2.9745
Epoch [33/200], Loss: 2.9388
Epoch [34/200], Loss: 2.8007
Epoch [35/200], Loss: 2

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Loss: 3.6834, Validation Accuracy: 33.63%
Validation Precision: 36.60%, Validation Recall: 33.63%
Test Loss: 3.9031, Test Accuracy: 33.08%
Test Precision: 40.16%, Test Recall: 33.08%
